In [1]:
import pandas as pd

# Import price file

In [2]:
columns = {
    'external ID': 'external_id',
    'Session fee.3': 'ac_session_price',
    'kwh.3': 'ac_price',
    'min.3': 'ac_time_price',
    'Fair Use [min].5': 'ac_fair_use_price',
    'Session fee.4': 'dc_session_price',
    'kwh.4': 'dc_price',
    'min.4': 'dc_time_price',
    'Fair Use [min].6': 'dc_fair_use_price'
}

raw = pd.read_csv('rigi_pricing.csv', header=1)
raw = raw.rename(columns=columns)[list(columns.values())]

raw.head(3)

,external_id,ac_session_price,ac_price,ac_time_price,ac_fair_use_price,dc_session_price,dc_price,dc_time_price,dc_fair_use_price
0,0,1.500 CHF,0.740 CHF,0.000 CHF,0,1.500 CHF,0.780 CHF,0.000 CHF,0
1,DEALL,1.500 CHF,0.540 CHF,0.000 CHF,0,1.500 CHF,0.540 CHF,0.000 CHF,0
2,IT*AXS,1.500 CHF,0.610 CHF,0.000 CHF,0,1.500 CHF,0.870 CHF,0.000 CHF,0


### Drop specific rows

In [3]:
# Investigate what rows should be eliminated
with pd.option_context('display.max_rows', None):  # more options can be specified also
    print(raw[['external_id','ac_price']])

   external_id   ac_price
0            0  0.740 CHF
1        DEALL  0.540 CHF
2       IT*AXS  0.610 CHF
3       IT*BEC  0.610 CHF
4       BE*BCA  0.610 CHF
5       DE*RBO  0.610 CHF
6       CZ*CEZ  0.610 CHF
7       DE*BDO  0.540 CHF
8       LU*CHY  0.760 CHF
9       DK*CLE  0.760 CHF
10       DECCH  0.820 CHF
11      DE*ISE  0.520 CHF
12      IT*DUF  0.720 CHF
13      DE*EDR  0.580 CHF
14      DE*ICT  0.520 CHF
15      CZ*EON  0.690 CHF
16      CH*ECU  0.740 CHF
17      DE*384  0.480 CHF
18      HU*ELM  0.490 CHF
19      IT*EMO  0.620 CHF
20      DE*EBW  0.530 CHF
21      IT*ELX  0.640 CHF
22      DE*ENC  0.480 CHF
23      AT*EST  0.530 CHF
24      CH*IBA  0.430 CHF
25      DE*WAY  0.640 CHF
26       ESGIC  0.650 CHF
27       CHEVP  0.840 CHF
28      NL*GFX  0.430 CHF
29      DE*GUC  0.570 CHF
30      AT*HTB  0.640 CHF
31      DE*730  0.060 CHF
32      IT*HRA  0.720 CHF
33       HR005  0.430 CHF
34      ES*IBD  0.650 CHF
35      ES*IBL  0.570 CHF
36      DE*VKW  0.560 CHF
37      DE*B

In [4]:
raw = raw.drop(0) # fallback price, added somewhere else
raw = raw.drop(63) # divider row
raw = raw.drop(70) # divider row

In [5]:
# Show all duplicate external ids
value_counts = raw['external_id'].value_counts(dropna=False)
duplicate_external_ids = list(value_counts[value_counts >= 2].index)
raw[raw['external_id'].isin(duplicate_external_ids)]

,external_id,ac_session_price,ac_price,ac_time_price,ac_fair_use_price,dc_session_price,dc_price,dc_time_price,dc_fair_use_price
27,CHEVP,1.500 CHF,0.840 CHF,0.000 CHF,0,1.500 CHF,0.970 CHF,0.000 CHF,0
64,DE*SNH,1.500 CHF,0.690 CHF,0.000 CHF,0,1.500 CHF,0.800 CHF,0.000 CHF,0
66,DE*SNH,2.700 CHF,0.490 CHF,0.000 CHF,0,2.700 CHF,0.650 CHF,0.000 CHF,0
68,CHEVP,1.500 CHF,0.540 CHF,0.000 CHF,0,1.500 CHF,0.540 CHF,0.000 CHF,0
69,CH*AMA,1.500 CHF,0.540 CHF,0.000 CHF,0,1.500 CHF,0.540 CHF,0.200 CHF,60
71,CH*AMA,1.500 CHF,0.450 CHF,0.000 CHF,0,1.500 CHF,0.450 CHF,0.200 CHF,60


In [6]:
# Handled later in the script DE*SNH:
# 65 --> price for all sub cpos
# 63 --> price for the DE*SNH cpo (has the same id as external id)
snh_sub_cpo_price = raw.iloc[64]
raw = raw.drop(64)

raw = raw.drop(27) # take the other, lower CHEVP price
raw = raw.drop(69) # for zvolt

In [7]:
print('max value counts:', max(raw['external_id'].value_counts(dropna=False)))

max value counts: 1


In [8]:
raw[raw['external_id'] == '+49*861']

,external_id,ac_session_price,ac_price,ac_time_price,ac_fair_use_price,dc_session_price,dc_price,dc_time_price,dc_fair_use_price
40,+49*861,1.500 CHF,0.610 CHF,0.100 CHF,300,1.500 CHF,0.060 CHF,0.100 CHF,180


# Import external cpo id mapping

In [9]:
mapping = pd.read_csv('sub_cpo_mapping.csv')
mapping.apply(lambda col: col.str.strip()) # remove spaces
mapping['sub_cpo_id'].apply(lambda s: len(s)).value_counts(dropna=False)

invalid_rows = mapping[mapping['sub_cpo_id'].apply(lambda s: len(s) != 6)]

print('FILTERING OUT')
print('Sub cpo ids with length not 6:\n')
print(invalid_rows)

mapping = mapping.drop(invalid_rows.index)

print('\n', sum(mapping.duplicated()), 'duplicate rows found')

FILTERING OUT
Sub cpo ids with length not 6:

                                   name sub_cpo_id external_id
86                    chargeIT mobility    +49*822      DE*BDO
109  Compleo Charging Technologies GmbH    +49*809      DE*ISE
227                           HUJ-Nobil   +358*899     +49*899
228                           HUJ-Nobil    +46*899     +49*899
229                           HUJ-Nobil    +47*899     +49*899
327         Porsche Smart Mobility GmbH      AT911       DE911
328         Porsche Smart Mobility GmbH      BE911       DE911
329         Porsche Smart Mobility GmbH      CH911       DE911
330         Porsche Smart Mobility GmbH      DE911       DE911
331         Porsche Smart Mobility GmbH      ES911       DE911
332         Porsche Smart Mobility GmbH      FR911       DE911
333         Porsche Smart Mobility GmbH      GB911       DE911
334         Porsche Smart Mobility GmbH      IT911       DE911
335         Porsche Smart Mobility GmbH      LU911       DE911
336      

# Map External id to Sub-cpo ids

In [10]:
df = raw.merge(mapping, how='left', left_on='external_id', right_on='external_id')

### Fix stupidity

In [11]:
sl = df[df['sub_cpo_id'].isna()]
sl

,external_id,ac_session_price,ac_price,ac_time_price,ac_fair_use_price,dc_session_price,dc_price,dc_time_price,dc_fair_use_price,name,sub_cpo_id
222,DE*2GO,1.500 CHF,0.640 CHF,0.070 CHF,240,1.500 CHF,0.850 CHF,0.140 CHF,55,NaN,NaN
269,CH*GOF,1.500 CHF,0.540 CHF,0.000 CHF,0,1.500 CHF,0.540 CHF,0.000 CHF,0,NaN,NaN
271,CH*AMA,1.500 CHF,0.450 CHF,0.000 CHF,0,1.500 CHF,0.450 CHF,0.200 CHF,60,NaN,NaN


In [12]:
# These external ids are actually sub-cpos and as such obviously don't map to more sub-cpos
# Morons.

# These are the real prices and should overwrite similar entries
# We need to drop any other row with those external ids

In [13]:
# drop other row with same sub cpo id
others = df.loc[df.index.difference(sl.index)]

In [14]:
print('df length:', len(df))
print('to delete and replace with slice:', sum(others['sub_cpo_id'].isin(list(sl['external_id']))))
print('others length:', len(others))

df length: 272
to delete and replace with slice: 3
others length: 269


In [15]:
others_cleaned = others[~others['sub_cpo_id'].isin(sl['external_id'])]
print('others_cleaned length:', len(others_cleaned))
# ignore warning
sl['sub_cpo_id'] = sl['external_id']
df = pd.concat([others_cleaned, sl])

print('new length:', len(df))

others_cleaned length: 266
new length: 269


/var/folders/5c/qfcpw_j92yx_4lnydrx1b2vh0000gn/T/ipykernel_36504/4081462832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sl['sub_cpo_id'] = sl['external_id']


## Check that all sub CPO ids have length 6

In [16]:
df['sub_cpo_id'].apply(lambda s: len(s)).value_counts()

6    269
Name: sub_cpo_id, dtype: int64

In [17]:
snh_sub_cpo_price

external_id             DE*SNH
ac_session_price     2.700 CHF
ac_price             0.490 CHF
ac_time_price        0.000 CHF
ac_fair_use_price            0
dc_session_price     2.700 CHF
dc_price             0.650 CHF
dc_time_price        0.000 CHF
dc_fair_use_price            0
Name: 66, dtype: object

In [18]:
df[df['sub_cpo_id'] == snh_sub_cpo_price['external_id']]

,external_id,ac_session_price,ac_price,ac_time_price,ac_fair_use_price,dc_session_price,dc_price,dc_time_price,dc_fair_use_price,name,sub_cpo_id
223,DE*SNH,2.700 CHF,0.490 CHF,0.000 CHF,0,2.700 CHF,0.650 CHF,0.000 CHF,0,Stromnetz Hamburg GmbH,DE*SNH


# Stack AC and DC

In [19]:
def cond_map_to_dict_and_strip(l, affix):
    return {c:c.strip(affix) for c in cols if affix in c}

cols = list(columns.values())
ac_pricing_columns = cond_map_to_dict_and_strip(cols, 'ac_')
dc_pricing_columns = cond_map_to_dict_and_strip(cols, 'dc_')

ac_columns = list(ac_pricing_columns.values()) + ['sub_cpo_id']
dc_columns = list(dc_pricing_columns.values()) + ['sub_cpo_id']

ac = df.rename(columns=ac_pricing_columns)[ac_columns]
ac['plug_type'] = 'AC'

dc = df.rename(columns=dc_pricing_columns)[dc_columns]
dc['plug_type'] = 'DC'

stacked = pd.concat([ac, dc], axis=0)
stacked['currency'] = 'CHF'
stacked['group'] = 'RIGI_CUSTOMER'

col_rename = {'sub_cpo_id': 'cpo_id'}
col_drop = ['external_id']

stacked = stacked.rename(columns=col_rename)
#stacked = stacked.drop(columns=col_drop)

# format price columns
price_columns = ['session_price', 'price', 'time_price']
for col in price_columns:
    stacked[col] = stacked[col].str.replace('CHF', '').str.strip().astype('float')

# reorder columns
ordered_cols = ['cpo_id', 'session_price', 'price', 'time_price', 'fair_use_price', 'plug_type', 'currency']
stacked = stacked[ordered_cols]
stacked['group'] = 'RIGI_CUSTOMER'

stacked.head()

,cpo_id,session_price,price,time_price,fair_use_price,plug_type,currency,group
0,BEALLE,1.5,0.54,0.0,0,AC,CHF,RIGI_CUSTOMER
1,DEALLE,1.5,0.54,0.0,0,AC,CHF,RIGI_CUSTOMER
2,DKALLE,1.5,0.54,0.0,0,AC,CHF,RIGI_CUSTOMER
3,FRALLE,1.5,0.54,0.0,0,AC,CHF,RIGI_CUSTOMER
4,FRLPIE,1.5,0.54,0.0,0,AC,CHF,RIGI_CUSTOMER


## Export

In [20]:
stacked.to_csv('rigi_prices_production.csv', index=False)